### Домашнее задание_1

In [1]:
import pandas as pd
import numpy as np
from sklearn.pipeline import Pipeline, make_pipeline
from sklearn.base import BaseEstimator, TransformerMixin
# from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import  OneHotEncoder
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import roc_auc_score
# from sklearn.feature_extraction.text import TfidfVectorizer
# import itertools

# import matplotlib.pyplot as plt

In [2]:
pd.set_option('display.max_columns', 100)
pd.set_option('display.max_rows', 500)

In [3]:
df = pd.read_csv("./WA_Fn-UseC_-Telco-Customer-Churn.csv")
df.head()

,customerID,gender,SeniorCitizen,Partner,Dependents,tenure,PhoneService,MultipleLines,InternetService,OnlineSecurity,OnlineBackup,DeviceProtection,TechSupport,StreamingTV,StreamingMovies,Contract,PaperlessBilling,PaymentMethod,MonthlyCharges,TotalCharges,Churn
0,7590-VHVEG,Female,0,Yes,No,1,No,No phone service,DSL,No,Yes,No,No,No,No,Month-to-month,Yes,Electronic check,29.85,29.85,No
1,5575-GNVDE,Male,0,No,No,34,Yes,No,DSL,Yes,No,Yes,No,No,No,One year,No,Mailed check,56.95,1889.5,No
2,3668-QPYBK,Male,0,No,No,2,Yes,No,DSL,Yes,Yes,No,No,No,No,Month-to-month,Yes,Mailed check,53.85,108.15,Yes
3,7795-CFOCW,Male,0,No,No,45,No,No phone service,DSL,Yes,No,Yes,Yes,No,No,One year,No,Bank transfer (automatic),42.30,1840.75,No
4,9237-HQITU,Female,0,No,No,2,Yes,No,Fiber optic,No,No,No,No,No,No,Month-to-month,Yes,Electronic check,70.70,151.65,Yes


In [4]:
df.shape

(7043, 21)

In [5]:
df.columns

Index(['customerID', 'gender', 'SeniorCitizen', 'Partner', 'Dependents',
       'tenure', 'PhoneService', 'MultipleLines', 'InternetService',
       'OnlineSecurity', 'OnlineBackup', 'DeviceProtection', 'TechSupport',
       'StreamingTV', 'StreamingMovies', 'Contract', 'PaperlessBilling',
       'PaymentMethod', 'MonthlyCharges', 'TotalCharges', 'Churn'],
      dtype='object')

In [6]:
df.describe()

,SeniorCitizen,tenure,MonthlyCharges
count,7043.000000,7043.000000,7043.000000
mean,0.162147,32.371149,64.761692
std,0.368612,24.559481,30.090047
min,0.000000,0.000000,18.250000
25%,0.000000,9.000000,35.500000
50%,0.000000,29.000000,70.350000
75%,0.000000,55.000000,89.850000
max,1.000000,72.000000,118.750000


In [7]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7043 entries, 0 to 7042
Data columns (total 21 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   customerID        7043 non-null   object 
 1   gender            7043 non-null   object 
 2   SeniorCitizen     7043 non-null   int64  
 3   Partner           7043 non-null   object 
 4   Dependents        7043 non-null   object 
 5   tenure            7043 non-null   int64  
 6   PhoneService      7043 non-null   object 
 7   MultipleLines     7043 non-null   object 
 8   InternetService   7043 non-null   object 
 9   OnlineSecurity    7043 non-null   object 
 10  OnlineBackup      7043 non-null   object 
 11  DeviceProtection  7043 non-null   object 
 12  TechSupport       7043 non-null   object 
 13  StreamingTV       7043 non-null   object 
 14  StreamingMovies   7043 non-null   object 
 15  Contract          7043 non-null   object 
 16  PaperlessBilling  7043 non-null   object 


**1. Какое соотношение мужчин и женщин в представленном наборе данных?**

In [8]:
df['gender'].value_counts()

Male      3555
Female    3488
Name: gender, dtype: int64

**2. Какое количество уникальных значений у поля InternetService?**

In [9]:
df['InternetService'].unique()

array(['DSL', 'Fiber optic', 'No'], dtype=object)

**3. Выведите статистики по полю TotalCharges (median, mean, std).**

In [10]:
df['TotalCharges'].describe

<bound method NDFrame.describe of 0         29.85
1        1889.5
2        108.15
3       1840.75
4        151.65
         ...   
7038     1990.5
7039     7362.9
7040     346.45
7041      306.6
7042     6844.5
Name: TotalCharges, Length: 7043, dtype: object>

***Сначала необходимо тип данных object преобразовать в числовой:***

In [11]:
df['TotalCharges'] = pd.to_numeric(df['TotalCharges'], errors='coerce')

In [12]:
df['TotalCharges'].head()

0      29.85
1    1889.50
2     108.15
3    1840.75
4     151.65
Name: TotalCharges, dtype: float64

***Все статистики можно получить сразу:***

In [13]:
df['TotalCharges'].describe()

count    7032.000000
mean     2283.300441
std      2266.771362
min        18.800000
25%       401.450000
50%      1397.475000
75%      3794.737500
max      8684.800000
Name: TotalCharges, dtype: float64

***Либо по отдельности:***

In [14]:
df['TotalCharges'].median()

1397.475

In [15]:
df['TotalCharges'].mean()

2283.3004408418656

In [16]:
df['TotalCharges'].std()

2266.771361883145

**В чем странность того, что вы получили? (подсказка: смотреть нужно на тип данных).**

***Числовые значения в поле TotalCharges имели тип данных object и с ними нельзя было производить математические операции.***

**4. Сделайте замену значений поля PhoneService на числовые (Yes->1, No->0).**

In [17]:
df['PhoneService'].unique()

array(['No', 'Yes'], dtype=object)

In [18]:
df.replace(
    {'PhoneService':
     {'Yes': 1}
    },
    inplace=True)

In [19]:
df.replace(
    {'PhoneService':
     {'No': 0}
    },
    inplace=True)

In [20]:
df['PhoneService'].head()

0    0
1    1
2    1
3    0
4    1
Name: PhoneService, dtype: int64

In [21]:
df.head()

,customerID,gender,SeniorCitizen,Partner,Dependents,tenure,PhoneService,MultipleLines,InternetService,OnlineSecurity,OnlineBackup,DeviceProtection,TechSupport,StreamingTV,StreamingMovies,Contract,PaperlessBilling,PaymentMethod,MonthlyCharges,TotalCharges,Churn
0,7590-VHVEG,Female,0,Yes,No,1,0,No phone service,DSL,No,Yes,No,No,No,No,Month-to-month,Yes,Electronic check,29.85,29.85,No
1,5575-GNVDE,Male,0,No,No,34,1,No,DSL,Yes,No,Yes,No,No,No,One year,No,Mailed check,56.95,1889.50,No
2,3668-QPYBK,Male,0,No,No,2,1,No,DSL,Yes,Yes,No,No,No,No,Month-to-month,Yes,Mailed check,53.85,108.15,Yes
3,7795-CFOCW,Male,0,No,No,45,0,No phone service,DSL,Yes,No,Yes,Yes,No,No,One year,No,Bank transfer (automatic),42.30,1840.75,No
4,9237-HQITU,Female,0,No,No,2,1,No,Fiber optic,No,No,No,No,No,No,Month-to-month,Yes,Electronic check,70.70,151.65,Yes


**5. Сделайте замену пробелов в поле TotalCharges на np.nan и приведите поле к типу данных float32. 
Затем заполните оставшиеся пропуски значением 0 с помощью метода fillna у столбца. 
Снова выведите статистики и сравните с тем, что вы видели в вопросе 3.**

***Пропуски в значениях в поле TotalCharges были заменены на NaN одновременно со сменой типа object на float64. Чтобы убедиться посмотрим на строку № 489.Изначально значение в этом поле было пропущено.***

In [22]:
df.iloc[488]

customerID                         4472-LVYGI
gender                                 Female
SeniorCitizen                               0
Partner                                   Yes
Dependents                                Yes
tenure                                      0
PhoneService                                0
MultipleLines                No phone service
InternetService                           DSL
OnlineSecurity                            Yes
OnlineBackup                               No
DeviceProtection                          Yes
TechSupport                               Yes
StreamingTV                               Yes
StreamingMovies                            No
Contract                             Two year
PaperlessBilling                          Yes
PaymentMethod       Bank transfer (automatic)
MonthlyCharges                          52.55
TotalCharges                              NaN
Churn                                      No
Name: 488, dtype: object

***Приведем поле TotalCharges к типу данных float32 и заменим оставшиеся пропуски на 0.***

In [23]:
# df['TotalCharges'].astype('float32') 

In [24]:
df['TotalCharges'].fillna(0, inplace=True)

In [25]:
df['TotalCharges'].astype('float32') 

0         29.850000
1       1889.500000
2        108.150002
3       1840.750000
4        151.649994
           ...     
7038    1990.500000
7039    7362.899902
7040     346.450012
7041     306.600006
7042    6844.500000
Name: TotalCharges, Length: 7043, dtype: float32

In [26]:
df['TotalCharges'].median()

1394.55

In [27]:
df['TotalCharges'].mean()

2279.7343035638223

In [28]:
df['TotalCharges'].std()

2266.7944696890213

***Так как количество наблюдений в поле TotalCharge изменилось (увеличилось), изменились и значения median, mean, std.***

**6. Сделайте замену значений поля Churn на числовые (Yes -> 1, No - 0).**

In [29]:
df['Churn'].unique()

array(['No', 'Yes'], dtype=object)

In [30]:
df['Churn'] = df['Churn'].replace(to_replace='Yes', value=1)

In [31]:
df['Churn'] = df['Churn'].replace(to_replace='No', value=0)

In [32]:
df['Churn'].head()

0    0
1    0
2    1
3    0
4    1
Name: Churn, dtype: int64

**7. Сделайте замену значений полей StreamingMovies, StreamingTV, TechSupport 
на числовые (Yes -> 1, No -> 0, No internet service->0).**

In [33]:
df['StreamingMovies'] = df['StreamingMovies'].replace(to_replace='Yes', value=1)

In [34]:
df['StreamingMovies'] = df['StreamingMovies'].replace(to_replace='No', value=0)

In [35]:
df['StreamingMovies'] = df['StreamingMovies'].replace(to_replace='No internet service', value=0)

In [36]:
df['StreamingTV'] = df['StreamingTV'].replace(to_replace='Yes', value=1)

In [37]:
df['StreamingTV'] = df['StreamingTV'].replace(to_replace='No', value=0)

In [38]:
df['StreamingTV'] = df['StreamingTV'].replace(to_replace='No internet service', value=0)

In [39]:
df['TechSupport'] = df['TechSupport'].replace(to_replace='Yes', value=1)

In [40]:
df['TechSupport'] = df['TechSupport'].replace(to_replace='No', value=0)

In [41]:
df['TechSupport'] = df['TechSupport'].replace(to_replace='No internet service', value=0)

In [42]:
df.head()

,customerID,gender,SeniorCitizen,Partner,Dependents,tenure,PhoneService,MultipleLines,InternetService,OnlineSecurity,OnlineBackup,DeviceProtection,TechSupport,StreamingTV,StreamingMovies,Contract,PaperlessBilling,PaymentMethod,MonthlyCharges,TotalCharges,Churn
0,7590-VHVEG,Female,0,Yes,No,1,0,No phone service,DSL,No,Yes,No,0,0,0,Month-to-month,Yes,Electronic check,29.85,29.85,0
1,5575-GNVDE,Male,0,No,No,34,1,No,DSL,Yes,No,Yes,0,0,0,One year,No,Mailed check,56.95,1889.50,0
2,3668-QPYBK,Male,0,No,No,2,1,No,DSL,Yes,Yes,No,0,0,0,Month-to-month,Yes,Mailed check,53.85,108.15,1
3,7795-CFOCW,Male,0,No,No,45,0,No phone service,DSL,Yes,No,Yes,1,0,0,One year,No,Bank transfer (automatic),42.30,1840.75,0
4,9237-HQITU,Female,0,No,No,2,1,No,Fiber optic,No,No,No,0,0,0,Month-to-month,Yes,Electronic check,70.70,151.65,1


**8. Заполните пропуски в поле PhoneService значением 0.**

In [43]:
df['PhoneService'].fillna(0, inplace=True)

In [44]:
df['PhoneService'].value_counts()

1    6361
0     682
Name: PhoneService, dtype: int64

**8. Для нашего датасета оставьте только указанный ниже список полей, удалив все другие и выведите верхние 3 строки.**

In [45]:
columns = ['gender', 'tenure', 'PhoneService', 'TotalCharges', 
           'StreamingMovies', 'StreamingTV', 'TechSupport', 'Churn']
#Ваш код здесь
def select_columns(data_frame, column_names):
    new_frame = data_frame.loc[:, column_names]
    return new_frame

In [46]:
df_new = select_columns(df, columns)

In [47]:
df_new.head(3)

,gender,tenure,PhoneService,TotalCharges,StreamingMovies,StreamingTV,TechSupport,Churn
0,Female,1,0,29.85,0,0,0,0
1,Male,34,1,1889.50,0,0,0,0
2,Male,2,1,108.15,0,0,0,1


In [48]:
df_new.shape

(7043, 8)

In [49]:
df_new.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7043 entries, 0 to 7042
Data columns (total 8 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   gender           7043 non-null   object 
 1   tenure           7043 non-null   int64  
 2   PhoneService     7043 non-null   int64  
 3   TotalCharges     7043 non-null   float64
 4   StreamingMovies  7043 non-null   int64  
 5   StreamingTV      7043 non-null   int64  
 6   TechSupport      7043 non-null   int64  
 7   Churn            7043 non-null   int64  
dtypes: float64(1), int64(6), object(1)
memory usage: 440.3+ KB


**9. Разделите датасет на тренировочную и тестовую выборку.**

In [50]:
features = ['gender', 'tenure', 'PhoneService', 'TotalCharges', 'StreamingMovies', 'StreamingTV', 'TechSupport']
target = 'Churn'
#Ваш код здесь

y = df_new[target]
# X = df_new.drop('Churn', axis=1)
X = df_new[features]
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=0)

**10. соберите pipeline для поля gender (нужно разобраться и изучить 
https://scikit-learn.org/stable/modules/generated/sklearn.pipeline.Pipeline.html ) из классов 
ColumnSelector и OHEEncoder, которые уже написаны ниже заранее.**

In [51]:
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.pipeline import Pipeline

class ColumnSelector(BaseEstimator, TransformerMixin):
    """
    Transformer to select a single column from the data frame to perform additional transformations on
    """
    def __init__(self, key):
        self.key = key

    def fit(self, X, y=None):
        return self

    def transform(self, X):
        return X[self.key]
    
class NumberSelector(BaseEstimator, TransformerMixin):
    """
    Transformer to select a single column from the data frame to perform additional transformations on
    Use on numeric columns in the data
    """
    def __init__(self, key):
        self.key = key

    def fit(self, X, y=None):
        return self

    def transform(self, X):
        return X[[self.key]]
    
class OHEEncoder(BaseEstimator, TransformerMixin):
    def __init__(self, key):
        self.key = key
        self.columns = []

    def fit(self, X, y=None):
        self.columns = [col for col in pd.get_dummies(X, prefix=self.key).columns]
        return self

    def transform(self, X):
        X = pd.get_dummies(X, prefix=self.key)
        test_columns = [col for col in X.columns]
        for col_ in test_columns:
            if col_ not in self.columns:
                X[col_] = 0
        return X[self.columns]


In [52]:
# gender = #Ваш код здесь
gender = Pipeline([
     ('coolumn_selector', ColumnSelector(key='gender')), 
     ('ohe', OHEEncoder(key='gender')), 
],
verbose=True)

**11. Вызовите метод fit_transform у пайплайна gender и передайте туда нашу тренировочную выборку 
(пример по ссылке из документации 
https://scikit-learn.org/stable/modules/generated/sklearn.pipeline.Pipeline.html#sklearn.pipeline.Pipeline.fit)**

In [53]:
gender.fit_transform(X_train)

[Pipeline] .. (step 1 of 2) Processing coolumn_selector, total=   0.0s
[Pipeline] ............... (step 2 of 2) Processing ohe, total=   0.0s


,gender_Female,gender_Male
3296,1,0
6397,0,1
6043,0,1
5309,1,0
3000,0,1
...,...,...
4931,0,1
3264,1,0
1653,0,1
2607,0,1


**12. Здесь код писать уже не нужно (все сделано за вас). К полю tenure применяем StandardScaler (нормируем и центрируем). 
Ссылка - https://scikit-learn.org/stable/modules/generated/sklearn.preprocessing.StandardScaler.html
Вопрос - в каких случаях это может быть полезно?**

In [54]:
from sklearn.preprocessing import StandardScaler

tenure = Pipeline([
                ('selector', NumberSelector(key='tenure')),
                ('standard', StandardScaler())
            ])

**13. Напишите аналогичный (как для tenure) преобразователь поля TotalCharges.**

In [55]:
TotalCharges = Pipeline([
                ('selector', NumberSelector(key='TotalCharges')),
                ('standard', StandardScaler())
            ])

**Объединение всех "кубиков" очень легко сделать таким образом:**

In [56]:
from sklearn.pipeline import FeatureUnion

number_features = Pipeline([
                ('selector', ColumnSelector(key=['PhoneService',
                                                 'StreamingMovies', 'StreamingTV', 
                                                 'TechSupport']))
            ])

In [57]:
feats = FeatureUnion([('tenure', tenure),
                      ('TotalCharges', TotalCharges),
                      ('continuos_features', number_features),
                      ('gender', gender)])
feature_processing = Pipeline([('feats', feats)])

**На этом этапе что мы сделали:
написали преобразователь поля gender, который делает OHE кодирование
написали преобразователь для поля tenure, который нормирует и центрирует его
повторили п. 2 для поля TotalCharges
для всех остальных просто взяли признаки как они есть, без изменений.**

**У нас уже готов наш пайплайн, который преобразовывает признаки. Давайте обучим модель поверх него. 
В качестве модели возьмем RandomForestClassifier.**

In [58]:
from sklearn.ensemble import RandomForestClassifier

pipeline = Pipeline([
    ('features', feats),
    ('classifier', RandomForestClassifier(random_state=42)),
])

pipeline.fit(X_train, y_train)

[Pipeline] .. (step 1 of 2) Processing coolumn_selector, total=   0.0s
[Pipeline] ............... (step 2 of 2) Processing ohe, total=   0.0s


Pipeline(steps=[('features',
                 FeatureUnion(transformer_list=[('tenure',
                                                 Pipeline(steps=[('selector',
                                                                  NumberSelector(key='tenure')),
                                                                 ('standard',
                                                                  StandardScaler())])),
                                                ('TotalCharges',
                                                 Pipeline(steps=[('selector',
                                                                  NumberSelector(key='TotalCharges')),
                                                                 ('standard',
                                                                  StandardScaler())])),
                                                ('continuos_features',
                                                 Pipeline(steps=[('selector',
          

**14. Сделайте прогноз вероятности оттока для X_test с помощью нашего предобученного на предыдущем шаге пайплайна и 
убедитесь что вам возвращаются вероятности для 2 классов.**

In [59]:
preds = pipeline.predict_proba(X_test)
preds

array([[0.78, 0.22],
       [0.87, 0.13],
       [0.75, 0.25],
       ...,
       [1.  , 0.  ],
       [0.98, 0.02],
       [1.  , 0.  ]])

In [60]:
preds_1 = pipeline.predict(X_test)
preds_1

array([0, 0, 0, ..., 0, 0, 0], dtype=int64)

**15. Посчитайте метрики качества получившейся модели (roc_auc).**

***Для подсчета метрики roc_auc_score необходимо получить прогноз оттока не в виде вероятности, а как метки классов:***

In [61]:
from sklearn.metrics import roc_auc_score

#Ваш код здесь
roc_auc_score(y_test, preds_1)

0.6536680455394077

**Сохраним наш пайплайн:**

In [62]:
import dill
with open("model_RF.dill", "wb") as f:
    dill.dump(pipeline, f)